In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import os
os.chdir('/home/jaentrouble/adipose_train3')
print(os.getcwd())

/home/jaentrouble/adipose_train3


In [3]:
import imageio as io
import json

img_names = os.listdir('data/done')
img = []
img_name_dict = {}
for idx, name in enumerate(img_names):
    img.append(io.imread('data/done/'+name))
    img_name_dict[name] = idx

json_names = os.listdir('data/save')
data = []
for name in json_names:
    with open('data/save/'+name,'r') as j:
        data.extend(json.load(j))
for datum in data :
    datum['image'] = img_name_dict[datum['image']]

In [4]:
data_train = data[:2600]
data_val = data[2600:2900]
data_test = data[2900:]

In [5]:
!git pull
%autoreload

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 1.34 KiB | 686.00 KiB/s, done.
From https://github.com/jaentrouble/Adipose_tf_train3
   7f56d63..bdcc3e5  master     -> origin/master
Updating 7f56d63..bdcc3e5
Fast-forward
 model_trainer.py  | 31 ++++++++++++++++++++++++++++-
 view_result.ipynb | 58 ++++++++++++++++---------------------------------------
 2 files changed, 47 insertions(+), 42 deletions(-)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from model_trainer import get_model
from box_models import *
from encoder_models import *
from functools import partial

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus :
    try :
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [7]:
img_size=(400,320)
encoder_f = hr_5_3_0_gap
box_f = partial(dense_128_4_normoutpos, image_size=img_size)
savedfile_name = 'hr_5_3_0_d128_normoutpos'
target_epoch = 100

In [8]:
test_model = get_model(encoder_f, box_f, img_size)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


In [9]:
from pathlib import Path
savedfile_name = Path(savedfile_name)
weight_path = 'savedmodels' / savedfile_name / str(target_epoch)
test_model.load_weights(str(weight_path))

In [ ]:
from model_trainer import create_val_data

X_test, Y_test = create_val_data(img, data, img_size, )